In [1]:
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import fit_zer, remove_zernike
import numpy as np
import os  
from astropy.io import fits
from copy import deepcopy
import proper

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
filename = 'cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s_ncpa_%s_%s.fits'
tag = {'L': 'LM', 'N2': 'N'}
rep = {'L': 'rep_6_-0h30', 'N2': 'rep_5_-0h30'}
npupils = {'L': 285, 'N2': 119}
sigLFs = {'L': 13e-9, 'N2': 13e-9*2}
sigHFs = {'L': 3e-9, 'N2': 3e-9*2}
nzer = 100
freq = 3
G = 0.4
lag = 3

In [3]:
def load_ncpa(case, band):
    npupil = npupils[band]
    scao = fits.getdata('cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%(band, npupil))
    cbw = fits.getdata('cbw/20221006/ncpa/%s_%s.fits'%(tag[band], rep[band]))
    wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_%s_%s_WVonly.fits'%(band, npupil))
    pup = fits.getdata('cfull/mask_Cfull_20220512_%s_%s.fits'%(band, npupil))
    pup[pup < .5] = 0
    if case == 'cbw':
        ncpa = scao + cbw
    elif case == 'wv':
        ncpa = scao + wv
    else:
        ncpa = scao + cbw + wv
    return pup, ncpa

def load_zpols(pup, ncpa, ncpa_name, npupil):
    zpols_name = ncpa_name[:-5] + '_zpols_%s.fits'%nzer
    try:
        zpols = fits.getdata(zpols_name)
        print('    getdata ' + zpols_name)
    except FileNotFoundError:
        print('    writeto ' + zpols_name)
        zpols = multiCPU(fit_zer, posargs=[pup, npupil/2, nzer], 
            posvars=[ncpa], case='get zpols')
        fits.writeto(zpols_name, np.float32(zpols))
    return zpols

def load_zpols_integ(pup, ncpa, ncpa_name, npupil, sigLF, sigHF):
    zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_freq_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freq, G, lag, sigLF, sigHF)
    if os.path.isfile(zpols_integ_name):
        zpols_integ = fits.getdata(zpols_integ_name)
        print('  getdata ' + zpols_integ_name)
    else:
        print('  write to ' + zpols_integ_name)
        zpols = load_zpols(pup, ncpa, ncpa_name, npupil)
        zpols_integ = np.zeros(zpols.shape)
        nframes = len(zpols)
        # piston
        zpols_integ[:,0] = zpols[:,0]
        # tip-tilt, and higher modes
        for m, sig in zip([range(1,3), range(3,nzer)], [sigLF, sigHF]):
            for n in range(freq+lag, nframes, freq):
                error = np.mean(zpols[n-freq-lag:n-lag,m] - zpols_integ[n-1,m], 0) + np.random.normal(0, sig, (1, len(m))) 
                zpols_integ[n:n+freq,m] = zpols_integ[n-1,m] + G*error
        fits.writeto(zpols_integ_name, np.float32(zpols_integ))
    return zpols_integ

def save_ncpa_cube(case, band):
    npupil = npupils[band]
    ncpa_name = filename%(case, band, npupil)
    if os.path.isfile(ncpa_name):
        print('file already exists: ' + ncpa_name)
    else:
        print('write to ' + ncpa_name)
        pup, ncpa = load_ncpa(case, band)
        sigLF = sigLFs[band]
        sigHF = sigHFs[band]
        zpols_integ = load_zpols_integ(pup, ncpa, ncpa_name, npupil, sigLF, sigHF)
        wf = proper.prop_begin(1, 1, npupil, 1) # initial wavefront
        _, HSF = multiCPU(remove_zernike, multi_out=True, verbose=True,
            posargs=[deepcopy(wf), pup],
            posvars=[ncpa, zpols_integ])
        fits.writeto(ncpa_name, np.float32(HSF))

In [4]:
for case in ['cbw', 'wv', 'all']:
    for band in ['L', 'N2']:
        save_ncpa_cube(case, band)

file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_cbw_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_cbw_ncpa_N2_119.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_wv_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_wv_ncpa_N2_119.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_N2_119.fits
